# 1 - Beginning Workflows

In this lesson, we'll cover the basics of using atomate to run calculations. This will be a hands-on lesson where we dive into running a full workflows and break that down into components to understand how the various moving parts give us the ability to scale from 1 calculation to 10's of thousands.

# Building a workflow

To begin, we'll start by grabbing a structure from materials project using pymatgen and the MPRester interface we learned about in a previous course

In [1]:
from pymatgen import MPRester

mpr = MPRester()

struc = mpr.get_structure_by_material_id("mp-27")
print(struc)

Full Formula (Si1)
Reduced Formula: Si
abc   :   2.736139   2.736139   2.736139
angles:  60.000000  60.000000  60.000000
Sites (1)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Si      0    0    0


Now, let's construct a workflow using atomate to optimize this structure in DFT

In [2]:
from atomate.vasp.workflows.presets.core import wf_structure_optimization

In [3]:
wf = wf_structure_optimization(struc,{"DB_FILE": None})
print(wf)

Workflow object: (fw_ids: dict_keys([-1]) , name: Si)


Get some more information on the workflow

In [4]:
wf.as_dict()

{'created_on': datetime.datetime(2018, 8, 7, 16, 46, 46, 293787),
 'fws': [{'created_on': '2018-08-07T16:46:46.293548',
   'fw_id': -1,
   'name': 'Si-structure optimization',
   'spec': {'_tasks': [{'_fw_name': 'FileWriteTask',
      'files_to_write': [{'contents': '',
        'filename': 'FW--Si-structure_optimization'}]},
     {'_fw_name': '{{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}',
      'structure': {'@class': 'Structure',
       '@module': 'pymatgen.core.structure',
       'charge': None,
       'lattice': {'a': 2.7361386705337156,
        'alpha': 60.00000010462969,
        'b': 2.7361386686542817,
        'beta': 60.000000127351925,
        'c': 2.73613867,
        'gamma': 60.000000062179055,
        'matrix': [[2.3695656, 0.0, 1.36806933],
         [0.7898552, 2.23404787, 1.36806933],
         [0.0, 0.0, 2.73613867]],
        'volume': 14.484360157964268},
       'sites': [{'abc': [0.0, 0.0, 0.0],
         'label': 'Si',
         'species': [{'element': 'Si'

# Running with Fake VASP to simulate a DFT calculation

Due to a combination of licensing issues and just not being able to run this quickly on the jupyter server, we're going to simulate VASP running with a magic function. You will later learn about powerups, which let you modify a workflow. For this exercise we're going to use a powerup that will replace the normal VASP running functionality with something that just copies files we've prepared for you

In [5]:
from atomate.vasp.powerups import use_fake_vasp

## Lets do some work to get the path to fake VASP files

In [6]:
import os

currrent_path = os.path.curdir
relative_path = os.path.join(currrent_path, "../../mp_workshop/fake_vasp/Si_structure_opt")
absolute_path = os.path.abspath(relative_path)
print(absolute_path)

/Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/mp_workshop/fake_vasp/Si_structure_opt


In [7]:
wf = use_fake_vasp(wf, ref_dirs={"Si-structure optimization": absolute_path})
wf.as_dict()

{'created_on': datetime.datetime(2018, 8, 7, 16, 46, 46, 293787),
 'fws': [{'created_on': '2018-08-07T16:46:46.293548',
   'fw_id': -1,
   'name': 'Si-structure optimization',
   'spec': {'_tasks': [{'_fw_name': 'FileWriteTask',
      'files_to_write': [{'contents': '',
        'filename': 'FW--Si-structure_optimization'}]},
     {'_fw_name': '{{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}',
      'structure': {'@class': 'Structure',
       '@module': 'pymatgen.core.structure',
       'charge': None,
       'lattice': {'a': 2.7361386705337156,
        'alpha': 60.00000010462969,
        'b': 2.7361386686542817,
        'beta': 60.000000127351925,
        'c': 2.73613867,
        'gamma': 60.000000062179055,
        'matrix': [[2.3695656, 0.0, 1.36806933],
         [0.7898552, 2.23404787, 1.36806933],
         [0.0, 0.0, 2.73613867]],
        'volume': 14.484360157964268},
       'sites': [{'abc': [0.0, 0.0, 0.0],
         'label': 'Si',
         'species': [{'element': 'Si'

## Now we have to get ourself a LaunchPad so that we can submit this workflow to our database


Atomate uses Fireworks as its workflow engine. Fireworks hides the database with an object called a LaunchPad. This allows you to submit and query workflows from anywhere you have database access. We need to get ourselves a LaunchPad object so we can submit our workflow

In [8]:
from fireworks.core.launchpad import LaunchPad

lp = LaunchPad()

We can use the launchpad to add a workkflow to our database:

In [9]:
lp.add_wf(wf)

2018-08-07 09:46:46,382 INFO Added a workflow. id_map: {-1: 1}


{-1: 1}

# Monitoring Workflows

Fireworks lets you monitor the status of workflows and fireworks using both python and the command line. Let's start off by looking at the status of our workflow. For each bit of python code, i'll include a cell with a command line command using jupyter-notebook's '!' functionality. In practice, we use the command line tools quite a bit and will be emphasized in this notebook.

**Command Line Access in Jupyter**: Jupyter lets you running command line commands by prefacing them with the exclamation mark:

In [11]:
# Lets get workflows

def get_wflows():
    for wf_id in lp.get_wf_ids():
        for key,value in lp.get_wf_summary_dict(wf_id).items():
            print(key, ": ",value)
        print("\n")

get_wflows()

state :  READY
name :  Si
created_on :  2018-08-07 16:46:46.293000
updated_on :  2018-08-07 16:46:46.293000
states :  OrderedDict([('Si-structure optimization--1', 'READY')])
launch_dirs :  OrderedDict([('Si-structure optimization--1', [])])




This is how you get workflow information on the command line

In [12]:
!lpad get_wflows

{
    "state": "READY",
    "name": "Si--1",
    "created_on": "2018-08-07T16:46:46.293000",
    "states_list": "REA"
}


In [13]:
def get_fws():
    for fw_id in lp.get_fw_ids():
        fw = lp.get_fw_dict_by_id(fw_id)
        for prop in ["fw_id","updated_on","state","name"]:
            print(prop, ": ",fw[prop])

        print("\n")
        
get_fws()

fw_id :  1
updated_on :  2018-08-07T16:46:46.351026
state :  READY
name :  Si-structure optimization




This command line gets you the same information

In [14]:
!lpad get_fws

{
    "fw_id": 1,
    "created_on": "2018-08-07T16:46:46.293548",
    "updated_on": "2018-08-07T16:46:46.351026",
    "state": "READY",
    "name": "Si-structure optimization"
}


In [15]:
!lpad --help

usage: lpad [-h] [-o {json,yaml}] [-l LAUNCHPAD_FILE] [-c CONFIG_DIR]
            [--logdir LOGDIR] [--loglvl LOGLVL] [-s]
            {version,init,reset,add,check_wflow,get_launchdir,append_wflow,dump_wflow,add_scripts,get_fws,track_fws,rerun_fws,defuse_fws,pause_fws,reignite_fws,resume_fws,update_fws,get_wflows,defuse_wflows,pause_wflows,reignite_wflows,archive_wflows,delete_wflows,get_qids,cancel_qid,detect_unreserved,detect_lostruns,set_priority,webgui,recover_offline,forget_offline,admin,report,introspect}
            ...

A command line interface to FireWorks. For more help on a specific command,
type "lpad <command> -h".

positional arguments:
  {version,init,reset,add,check_wflow,get_launchdir,append_wflow,dump_wflow,add_scripts,get_fws,track_fws,rerun_fws,defuse_fws,pause_fws,reignite_fws,resume_fws,update_fws,get_wflows,defuse_wflows,pause_wflows,reignite_wflows,archive_wflows,delete_wflows,get_qids,cancel_qid,detect_unreserved,detect_lostruns,set_priority,webgui,recover_off

In [16]:
# Let's look at what this command can do:
!lpad get_fws --help

usage: lpad get_fws [-h] [-i FW_ID [FW_ID ...]] [-n NAME]
                    [-s {ARCHIVED,FIZZLED,DEFUSED,PAUSED,WAITING,READY,RESERVED,RUNNING,COMPLETED}]
                    [-q QUERY] [-lm] [--qid QID]
                    [-d {all,more,less,ids,count,reservations}] [-m MAX]
                    [--sort {created_on,updated_on}]
                    [--rsort {created_on,updated_on}]

optional arguments:
  -h, --help            show this help message and exit
  -i FW_ID [FW_ID ...], --fw_id FW_ID [FW_ID ...]
                        fw_id
  -n NAME, --name NAME  get FWs with this name
  -s {ARCHIVED,FIZZLED,DEFUSED,PAUSED,WAITING,READY,RESERVED,RUNNING,COMPLETED}, --state {ARCHIVED,FIZZLED,DEFUSED,PAUSED,WAITING,READY,RESERVED,RUNNING,COMPLETED}
                        Select by state.
  -q QUERY, --query QUERY
                        Query (enclose pymongo-style dict in single-quotes,
                        e.g. '{"state":"COMPLETED"}')
  -lm, --launches_mode  Query the launches colle

# Now lets run this workflow


There are a few different ways to run a workflow. The first is to just run it within this notebook directly.

In [17]:
from fireworks.core.rocket_launcher import launch_rocket

In [18]:
# Lets move into a temporary working directory
import os

os.mkdir("temp")
os.chdir("temp")

In [19]:
launch_rocket(lp)

2018-08-07 09:49:02,660 INFO Launching Rocket
2018-08-07 09:49:02,737 INFO RUNNING fw_id: 1 in directory: /Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/lessons/atomate/temp
2018-08-07 09:49:02,745 INFO Task started: FileWriteTask.
2018-08-07 09:49:02,747 INFO Task completed: FileWriteTask 
2018-08-07 09:49:02,750 INFO Task started: {{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}.
2018-08-07 09:49:02,764 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}} 
2018-08-07 09:49:02,767 INFO Task started: {{atomate.vasp.firetasks.run_calc.RunVaspFake}}.
2018-08-07 09:49:02,790 INFO atomate.vasp.firetasks.run_calc RunVaspFake: verified inputs successfully
2018-08-07 09:49:02,821 INFO atomate.vasp.firetasks.run_calc RunVaspFake: ran fake VASP, generated outputs
2018-08-07 09:49:02,822 INFO Task completed: {{atomate.vasp.firetasks.run_calc.RunVaspFake}} 
2018-08-07 09:49:02,826 INFO Task started: {{atomate.common.firetasks.glue_tasks.PassCa

/Users/shyamd/codes/pymatgen/pymatgen/command_line/bader_caller.py:392: UserWarning: Could not find AECCAR0, interpret Bader results with caution.
  warnings.warn(warning)
/Users/shyamd/codes/pymatgen/pymatgen/command_line/bader_caller.py:392: UserWarning: Could not find AECCAR2, interpret Bader results with caution.
  warnings.warn(warning)


2018-08-07 09:49:07,367 INFO atomate.vasp.drones Post-processing dir:/Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/lessons/atomate/temp
2018-08-07 09:49:07,370 WARNING atomate.vasp.drones Transformations file does not exist.
2018-08-07 09:49:07,373 INFO atomate.vasp.drones Post-processed /Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/lessons/atomate/temp
2018-08-07 09:49:07,376 INFO Task completed: {{atomate.vasp.firetasks.parse_outputs.VaspToDb}} 
2018-08-07 09:49:07,396 INFO Rocket finished


True

Now, lets see how that changed our fireworks

In [20]:
!lpad get_fws

{
    "fw_id": 1,
    "created_on": "2018-08-07T16:46:46.293548",
    "updated_on": "2018-08-07T16:49:07.390173",
    "state": "COMPLETED",
    "name": "Si-structure optimization"
}


This let me run a single firework in the notebook. What if I wanted to run multiple fireworks? First lets reset the old firework and add some more workflows to our database

In [21]:
# Let's use the python to rerun those fireworks
for fw_id in lp.get_fw_ids():
    lp.rerun_fw(fw_id)
    

In [22]:
# We can do the same thing using the command line:
!lpad rerun_fws 

2018-08-07 09:50:06,391 INFO Finished setting 1 FWs to rerun


In [23]:
!lpad get_fws

{
    "fw_id": 1,
    "created_on": "2018-08-07T16:46:46.293548",
    "updated_on": "2018-08-07T16:50:06.387406",
    "state": "READY",
    "name": "Si-structure optimization"
}


In [24]:
# Let's add the workflow a few more times to have multiple fireworks in database
lp.add_wf(wf)
lp.add_wf(wf)

2018-08-07 09:50:19,780 INFO Added a workflow. id_map: {1: 2}
2018-08-07 09:50:19,784 INFO Added a workflow. id_map: {2: 3}


{2: 3}

We can run all of the available fireworks using a 2 lines of python and a single command:

In [25]:
from fireworks.core.rocket_launcher import rapidfire
rapidfire(lp)

2018-08-07 09:50:35,046 INFO Created new dir /Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/lessons/atomate/temp/launcher_2018-08-07-16-50-35-046060
2018-08-07 09:50:35,046 INFO Launching Rocket
2018-08-07 09:50:35,069 INFO RUNNING fw_id: 1 in directory: /Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/lessons/atomate/temp/launcher_2018-08-07-16-50-35-046060
2018-08-07 09:50:35,079 INFO Task started: FileWriteTask.
2018-08-07 09:50:35,081 INFO Task completed: FileWriteTask 
2018-08-07 09:50:35,084 INFO Task started: {{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}.
2018-08-07 09:50:35,094 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}} 
2018-08-07 09:50:35,098 INFO Task started: {{atomate.vasp.firetasks.run_calc.RunVaspFake}}.
2018-08-07 09:50:35,120 INFO atomate.vasp.firetasks.run_calc RunVaspFake: verified inputs successfully
2018-08-07 09:50:35,142 INFO atomate.vasp.firetasks.run_calc RunVaspFake: ran fake VAS

/Users/shyamd/codes/pymatgen/pymatgen/command_line/bader_caller.py:392: UserWarning: Could not find AECCAR0, interpret Bader results with caution.
  warnings.warn(warning)
/Users/shyamd/codes/pymatgen/pymatgen/command_line/bader_caller.py:392: UserWarning: Could not find AECCAR2, interpret Bader results with caution.
  warnings.warn(warning)


2018-08-07 09:50:39,643 INFO atomate.vasp.drones Post-processing dir:/Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/lessons/atomate/temp/launcher_2018-08-07-16-50-35-046060
2018-08-07 09:50:39,645 WARNING atomate.vasp.drones Transformations file does not exist.
2018-08-07 09:50:39,649 INFO atomate.vasp.drones Post-processed /Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/lessons/atomate/temp/launcher_2018-08-07-16-50-35-046060
2018-08-07 09:50:39,653 INFO Task completed: {{atomate.vasp.firetasks.parse_outputs.VaspToDb}} 
2018-08-07 09:50:39,674 INFO Rocket finished
2018-08-07 09:50:39,680 INFO Created new dir /Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/lessons/atomate/temp/launcher_2018-08-07-16-50-39-679976
2018-08-07 09:50:39,680 INFO Launching Rocket
2018-08-07 09:50:39,699 INFO RUNNING fw_id: 2 in directory: /Users/shyamd/Dropbox/Projects/2018 - MP Workshop/workshop-2018/lessons/atomate/temp/launcher_2018-08-07-16-50-39-679976
20

This let us run fireworks until we no longer had any to run. But we're still running fireworks in our jupyter notebook. If I want to run on this on another machine I need to do something else. Normally, we would want to launch these jobs to our supercomputing queue and let that run them as resources become available. 

### Using the queue launcher:

Setting up the queue launcher unfortunately takes some work. There are configuration files to tell atomate how to submit jobs, where the database is and what special parameters to use for this supercomputer. 

This has all been setup for you in this workshop. Once setup, to use the queue, we simply launch the fireworks to the queue.

In [26]:
#TODO: Setting up Fireworks for launching queue

In [ ]:
!qlaunch rapidfire

Now, the supercomputer will take care of running the jobs and eventually we can test to see that they are working

In [ ]:
!lpad get_fws